In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
sliderDL = pd.read_csv('DraftLeagueSlider.csv')
sliderMLB = pd.read_csv('MLBSlider.csv')

In [3]:
sliderMLB = sliderMLB[sliderMLB['pitches'] > 50]
sliderMLB.drop('pitches', inplace=True, axis=1)

In [4]:
sliderDL['league'] = 'Draft League'
sliderMLB['league'] = 'MLB'

In [5]:
sliderDL = sliderDL.astype(str)
sliderMLB = sliderMLB.astype(str)

In [6]:
pitchersDL = list(set(sliderDL['pitcher']))

In [7]:
slider = pd.concat([sliderDL, sliderMLB])

In [8]:
def combined_features(row):
    return row['avgvelo']+" "+row['avgspinrate']+" "+row['avghorzbreak']+" "+row['avgvertbreak']+" "+row['avgeffectvelo']

In [9]:
#fig, ax = plt.subplots(figsize=(100,100))
#sns.heatmap(cosine_sim,cmap='magma',ax=ax)

In [10]:
def get_index_from_pitcher(pitcher):
    return slider[slider.pitcher == pitcher].index[0]

In [11]:
def get_pitcher_from_index(index):
    return slider[slider.index == index].values[0][0]

In [12]:
mlb_draft_pitcher = []
mlb_comp = []
cos_value = []

for x in pitchersDL:
    
    mlb_draft_pitcher.append(x)
    
    try:

        slider = pd.concat([sliderDL, sliderMLB])

        slider["combined_features"] = slider.apply(combined_features, axis =1)

        cv = CountVectorizer()
        count_matrix = cv.fit_transform(slider["combined_features"])

        cosine_sim = cosine_similarity(count_matrix)

        pitcher_like = x
        slider = slider[(slider['pitcher'] == pitcher_like) | (slider['league'] == 'MLB')]

        player_index = get_index_from_pitcher(pitcher_like)

        similar_pitchers = list(enumerate(cosine_sim[player_index]))

        sorted_similar_pitchers = sorted(similar_pitchers, key=lambda x:x[1], reverse=True)

        mlb_comp.append(get_pitcher_from_index(sorted_similar_pitchers[1][0]))
        
        cos_value.append(sorted_similar_pitchers[1][1])
                
    except IndexError:
        mlb_comp.append('No Comp')
        cos_value.append('No Similarity')
        pass

In [13]:
cosine_comp = pd.DataFrame(list(zip(mlb_draft_pitcher, mlb_comp, cos_value)), columns=['MLB DL Pitcher', 'MLB Comp', "Cosine Value"])
cosine_comp = cosine_comp.sort_values('MLB DL Pitcher')
cosine_comp

,MLB DL Pitcher,MLB Comp,Cosine Value
35,"Alba, Max","McGowin, Kyle",0.333333
20,"Alba, Miguel",No Comp,No Similarity
98,"Allain, Luke","Magill, Matt",0.316228
107,"Ammerlaan, Ashe",No Comp,No Similarity
15,"Ayers, Kyle","Williams, Taylor",0.251976
...,...,...,...
27,"Wilson, Benjamin","Lowther, Zac",0.286039
75,"Wilson, Jeffrey","Britton, Zack",0.235702
116,"Wiseman, Kevin","Britton, Zack",0.235702
29,"Wright, Kerry","Jax, Griffin",0.333333


In [14]:
cosine_comp.to_csv('slider_cosine_comp.csv', sep='\t')